In [16]:
#importing required packages
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim
from torchvision import datasets, transforms
import os
import numpy as np
import pandas as pd


In [14]:
#Hyperparamaters
batch_size = 100
sequence_len= 28
input_len = 28
num_layers = 2
num_class = 10
hidden_size = 128
epochs = 5
learning_rate = 0.01

In [5]:
#preparing datasets
train_data = datasets.FashionMNIST(root= './datasets/fashion_mnist', train=True, transform=transforms.ToTensor, download=True)
test_data = datasets.FashionMNIST(root= "./datasets/fashion_mnist", train=False, transform=transforms.ToTensor)

train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)



0.9%

100.0%


Extracting ./datasets/fashion_mnist\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./datasets/fashion_mnist\FashionMNIST\raw


100.0%


Extracting ./datasets/fashion_mnist\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./datasets/fashion_mnist\FashionMNIST\raw




25.2%

100.0%


Extracting ./datasets/fashion_mnist\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./datasets/fashion_mnist\FashionMNIST\raw



100.0%

Extracting ./datasets/fashion_mnist\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./datasets/fashion_mnist\FashionMNIST\raw



In [9]:
class LSTM(nn.Module):
    def __init__(self, input_len, hidden_size, num_class, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_len, hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, num_class)
    
    def forward(self, X):
        hidden_states = torch.zeros(self, num_layers, X.size[0], self.hidden_size)
        cell_states = torch.zeroes(self.num_layers, X.size(0), self.hidden_size)
        out,_ = self.lstm(X, (hidden_states, cell_states))
        out = self.output_layer(out[:, -1,:])

In [15]:
model = LSTM(input_len, hidden_size, num_class, num_layers)
print(model)

LSTM(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=10, bias=True)
)


In [18]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
def training(epochs, model, train_dataloader):
    total_steps = len(train_dataloader)
    
    for epoch in range(epochs):
        for batch, (image, labels) in enumerate(train_dataloader):
            images = images.reshape(-1, sequence_len, input_len)
            